In [ ]:
# libraries
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
dir = os.getcwd()
file_path = os.path.join(dir,"DataSets-main" )
cab_df = pd.read_csv(file_path + "/Cab_Data.csv")
city_df = pd.read_csv(file_path + "/City.csv")
customer_df = pd.read_csv(file_path + "/Customer_ID.csv")
transaction_df = pd.read_csv(file_path + "/Transaction_ID.csv")

# change the date format to a more readable format
cab_df['Date of Travel'] = pd.to_datetime(cab_df['Date of Travel'], origin='1899-12-30', unit='D')

In [ ]:
# Get information of the cab dataframe
cab_df.info()
# Percentage of missing values
print("Percentage of missing values")
cab_df.isnull().sum()/ len(cab_df)

In [ ]:
city_df.head()


In [ ]:
city_df.info()

#Checking for null values
city_df.isna().sum() / len(city_df)

In [ ]:
customer_df.head()


In [ ]:
customer_df.info()

customer_df.isnull().sum()

In [ ]:
transaction_df.head()

In [ ]:
transaction_df.info()

transaction_df.isnull().sum()

## Combining the datasets to one dataframe

In [ ]:
# Customer information
customer_info = pd.merge(customer_df, transaction_df)
customer_info.head()


In [ ]:
# Cab information
cab_info = pd.merge(cab_df, city_df)
cab_info.head()

In [ ]:
#print the shapes of the initial dataframes
print(f"Shape for cab: {cab_df.shape}, customer : {customer_df.shape}, City: {city_df.shape}, transaction: {transaction_df.shape}")

### Encoding categorical columns

In [ ]:
# Select the columns to be encoded
target_columns = customer_info[['Gender', 'Payment_Mode']]

# Perform label encoding
label_encoder = LabelEncoder()
label_encoded = target_columns.apply(label_encoder.fit_transform)

# Perform one-hot encoding
one_hot_encoder = OneHotEncoder()
one_hot_encoded = one_hot_encoder.fit_transform(label_encoded)

# Get the feature names
encoded_feature_names = []
for col in target_columns.columns:
    # Getting the unique categorical values in the target columns
    unique_values = target_columns[col].unique()
    for value in unique_values:
        encoded_feature_names.append(f"{col}_{value}")

# create a dataframe of the one-hot encoded columns
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoded_feature_names)

# Concatenate the customer dataframe with the one-hot encoded columns
customer_info_encoded = pd.concat([customer_info.drop(target_columns, axis=1), one_hot_encoded_df], axis=1)

one_hot_encoded_df.head()


## Understanding the relationships between the data

In [ ]:
# combined data ---> Actionable data. It contains data of customers that have used the cab between 31/01/2016 to 31/12/2018
combined_df = customer_info_encoded.merge(cab_info, on='Transaction ID')
combined_df.head()

### 1. Market share for each company
   -  It shows the level of acceptance or popularity of the cabs in the market.
   - The market share of each company is calculated by comparing the total number of trips made by each company to the total number of trips in the market

In [ ]:
# Filter the dataframe based on Company
pink_cab_df = combined_df[(combined_df['Company'] == 'Pink Cab')]
yellow_cab_df = combined_df[(combined_df['Company'] == 'Yellow Cab')]

# Total number of trips for each company
pink_cab_trips = pink_cab_df.shape[0]
yellow_cab_trips = yellow_cab_df.shape[0]

# Total number of trips in the market
total_trips = combined_df.shape[0]

# Market share for each company
pink_cab_market_share = pink_cab_trips / total_trips * 100
yellow_cab_market_share = yellow_cab_trips / total_trips * 100

# Pie chart to visualize the market share
labels = ['Pink Cab', 'Yellow Cab']
sizes = [pink_cab_market_share, yellow_cab_market_share]
colors = ['r', 'b']
explode = (0.1, 0)

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Market Share: Pink Cab vs. Yellow Cab')
plt.show()

### 2. Customer Retention
   -  Customer retention is crucial for the long-term success of a cab company.

In [ ]:
# Determine the time frames within the specified time period
start_date = '2016-01-31'
end_date = '2018-12-31'
time_frames = pd.date_range(start=start_date, end=end_date, freq='Y')

# Calculate the customer retention rate for each company for each time frame
retention_rates_pink_cab = []
retention_rates_yellow_cab = []

for i in range(len(time_frames) - 1):
    start_date = time_frames[i].strftime('%Y-%m-%d')
    end_date = time_frames[i+1].strftime('%Y-%m-%d')

    #determine the start amd end customers of tje pink cab
    pink_cab_start_customers = len(pink_cab_df[pink_cab_df['Date of Travel'] <= start_date]['Customer ID'].unique())
    pink_cab_end_customers = len(pink_cab_df[pink_cab_df['Date of Travel'] <= end_date]['Customer ID'].unique())

    # determine the new customers throughout the year
    pink_cab_new_customers = len(pink_cab_df[(pink_cab_df['Date of Travel'] >= start_date) & (pink_cab_df['Date of Travel'] <= end_date)]['Customer ID'].unique())

    # determine the start and end customers pf tje pink cab
    yellow_cab_start_customers = len(yellow_cab_df[yellow_cab_df['Date of Travel'] <= start_date]['Customer ID'].unique())
    yellow_cab_end_customers = len(yellow_cab_df[yellow_cab_df['Date of Travel'] <= end_date]['Customer ID'].unique())

    # determine the new customers throughout the year
    yellow_cab_new_customers = len(yellow_cab_df[(yellow_cab_df['Date of Travel'] >= start_date) & (yellow_cab_df['Date of Travel'] <= end_date)]['Customer ID'].unique())

    # calculating the retention rate
    pink_cab_retention_rate = (pink_cab_end_customers - pink_cab_new_customers) / pink_cab_start_customers * 100
    yellow_cab_retention_rate = (yellow_cab_end_customers - yellow_cab_new_customers) / yellow_cab_start_customers * 100

    retention_rates_pink_cab.append(pink_cab_retention_rate)
    retention_rates_yellow_cab.append(yellow_cab_retention_rate)


for i in range(len(time_frames) - 1):
    year = time_frames[i].year
    pink_cab_rate = retention_rates_pink_cab[i]
    yellow_cab_rate = retention_rates_yellow_cab[i]

    print(f"Retention Rates for Year {year}:")
    print("Pink Cab: {:.2f}%".format(pink_cab_rate))
    print("Yellow Cab: {:.2f}%".format(yellow_cab_rate))
    print()

# Plotting retention rates for Pink Cab
plt.plot(time_frames[:-1], retention_rates_pink_cab, marker='o', label='Pink Cab')

# Plotting retention rates for Yellow Cab
plt.plot(time_frames[:-1], retention_rates_yellow_cab, marker='o', label='Yellow Cab')

# Set title and labels
plt.title('Customer Retention Rates Over Time')
plt.xlabel('Year')
plt.ylabel('Retention Rate (%)')

# Set x-axis tick labels
plt.xticks(time_frames[:-1], [str(year) for year in time_frames[:-1].year], rotation=45)

# Add legend
plt.legend()

# Display the plot
plt.show()

### 3. Customer Segmentation

In [ ]:
cab_distribution = combined_df.groupby(['City', 'Company']).size().unstack(level=1)

# Stacked bar graph for cab distribution in all cities
cab_distribution.plot(kind='bar', figsize=(12, 6))
plt.xlabel('City')
plt.ylabel('Number of Trips')
plt.title('Cab Distribution in All Cities')
plt.legend()
plt.show()

# Group the data by city and cab company and count the number of unique customer IDs
customer_distribution = combined_df.groupby(['City', 'Company'])['Customer ID'].nunique().unstack()
# Create a bar graph to visualize the customer distribution or presence in each city for the cab companies
customer_distribution.plot(kind='bar', figsize=(12, 6))
plt.xlabel('City')
plt.ylabel('Number of Customers')
plt.title('Customer Distribution in Each City for Cab Companies')
plt.legend()
plt.show()

## 4. Customer Preferences

* Determining customer preference based on gender, payment methods and age

In [1]:
# Define the factors to consider for customer preferences
gender_factors = ['Gender_Male', 'Gender_Female']
payment_factors = ['Payment_Mode_Card', 'Payment_Mode_Cash']
age_factor = 'Age'

# Customer preferences for each factor and cab company
pink_cab_preferences = pink_cab_df.groupby('City')[gender_factors + payment_factors + [age_factor]].sum()
pink_cab_preferences_age = pink_cab_df.groupby('City')[gender_factors + [age_factor]]
yellow_cab_preferences = yellow_cab_df.groupby('City')[gender_factors + payment_factors + [age_factor]].sum()

# Creating separate graphs for gender preferences, payment preferences, and age preferences for each cab company
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 14))

# Gender Preferences
pink_cab_preferences[gender_factors].plot(kind='bar', ax=axes[0, 0], title='Gender Preferences: Pink Cab')
yellow_cab_preferences[gender_factors].plot(kind='bar', ax=axes[0, 1], title='Gender Preferences: Yellow Cab')

# Payment Preferences
pink_cab_preferences[payment_factors].plot(kind='bar', ax=axes[1, 0], title='Payment Preferences: Pink Cab')
yellow_cab_preferences[payment_factors].plot(kind='bar', ax=axes[1, 1], title='Payment Preferences: Yellow Cab')

# Age Preferences
# pink_cab_preferences[age_factor].plot(kind='bar', ax=axes[2, 0], title='Age Preferences: Pink Cab')
# yellow_cab_preferences[age_factor].plot(kind='bar', ax=axes[2, 1], title='Age Preferences: Yellow Cab')
# pink_cab_preferences_age.plot(kind='bar', ax=axes[2, 0], title='Age Preferences: Pink Cab')
yellow_cab_preferences[age_factor].plot(kind='bar', ax=axes[2, 1], title='Age Preferences: Yellow Cab')

# Adjust the layout and spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()


NameError: name 'pink_cab_df' is not defined

In [ ]:
pink_cab_preferences_age = pink_cab_df.groupby('City')[gender_factors + [age_factor]]

In [ ]:
print(pink_cab_preferences_age.head())

In [ ]:
# Calculate profits per km for each trip
combined_df['Profit per KM'] = (combined_df['Price Charged'] - combined_df['Cost of Trip']) / combined_df['KM Travelled']

# Print the resulting dataframe with profits per km
print(combined_df[['Company', 'City', 'KM Travelled', 'Price Charged', 'Cost of Trip', 'Profit per KM']])


### 4. Profits per kilometer

In [ ]:
# Calculate profits per kilometer for each cab company
combined_df['Profit per KM'] = (combined_df['Price Charged'] - combined_df['Cost of Trip']) / combined_df['KM Travelled']

# Group the data by cab company and calculate the average profit per kilometer
profits_per_km = combined_df.groupby('Company')['Profit per KM'].mean()

# Create a bar graph to visualize the profits per kilometer for each cab company
profits_per_km.plot(kind='bar', figsize=(12, 6))
plt.xlabel('Cab Company')
plt.ylabel('Profit per Kilometer (USD)')
plt.title('Profits per Kilometer for Each Cab Company')
plt.show()


### 5. Demand for days of the week

In [ ]:

# Convert the "Date of Travel" column to datetime format
combined_df['Date of Travel'] = pd.to_datetime(combined_df['Date of Travel'])

# Extract the day of the week
combined_df['Day of Week'] = combined_df['Date of Travel'].dt.day_name()

# Group the data by cab company and day of the week, and calculate the total number of trips
trips_per_day = combined_df.groupby(['Company', 'Day of Week']).size().unstack(level=1)

# Create visualizations for each cab company to compare the productivity of each day and assess the impact of holidays
cab_companies = combined_df['Company'].unique()

for company in cab_companies:
    trips_company = trips_per_day.loc[company]
    holidays = ['Saturday', 'Sunday']  # Define the holidays

    # Bar graph to compare the productivity of each day
    trips_company.plot(kind='bar', figsize=(12, 6))
    plt.xlabel('Day of Week')
    plt.ylabel('Number of Trips')
    plt.title(f'Productivity of Each Day for {company}')
    plt.show()




# Recommendation

## After evaluating both the cab companies on following points and found Yellow cab better than Pink cab:
- Market Share -> Yellow Cab has a higher market share of 76.4% throughout the whole cities as compared to 23.6% for Pink cab.
- Customer Retention -> Customer retention is crucial for the long-term success of the cab company. In 2016, Pink Cab had a higher retention rate at 46.28% as compared to Yelow Cab at 26.37%. IN 2017, Pink cab increased its customer retnetion to 52.46% as compared to 35.56% fro Yellow Cab. In general, Pink Cab has had a better customer retention rate throughout its operation period.
- Customer Segmentaion -> Yellow Cab is the most favoured in all cities except NASHVILLE TN, PITTSBURGH PA, SACRAMENTO CA, and SAN DIEGO CA where Pionk cab had an upper hand.
- Customer Preferences -> Most customers prefer to pay with card as compared to cash payment. The female gender are the majority users of the cabs for both cabs however, more men preferred to use the Pink Cab as compared to Yellow cab.